# 在Plotly图表与运算核之间通信

上节介绍的方法所绘制的图表一旦创建就无法再通过Python修改其数据或各种显示属性。本节利用前面两节介绍的方法实现Plotly图表与运算核之间的通信，通过本节介绍的方法，我们可以在图表创建之后：

* 动态更新曲线的数据或者图表的任何显示属性。
* 将图表上发生的交互事件发送给运算核，例如可以实现数据的点选。

In [1]:
from plotlyhelp import init_plotly_offline_mode
init_plotly_offline_mode()

为了实现Plotly图表与运算核之间的通信，我们需要同时编写在客户端运行的Javascript程序和在运算核中运行的Python程序。这两个程序之间通信需要使用事先约定好的频道。

下面是生成Javascript的模板，它有如下参数：

* `uid`:页面中表示图表的元素的标识，该标识也用于通信的频道名。
* `onclick`:是否将图表的点击事件(`plotly_click`)发送给运算核。
* `onrelayout`:是否将图表的布局事件(`plotly_relayout`)发送给运算核。该事件在X-Y轴的显示范围改变时触发。

当运算核开启`uid`频道的通信时，由`register_target()`注册的回调函数将被运行。在`msg.content.data`中保存着由运算核发来的数据，其中`code`键对应的是需要被运行的代码，而`data`键对应的是数据。这是使用`eval()`动态执行代码，代码中`graph`变量为页面上表示图表的元素，`data`为传递过来的数据。使用这种方法，可以通过发送不同的代码控制图表的各个不同部分的显示。

In [2]:
from jinja2 import Template

JsTemplate = Template("""
<script>
(function(){
var comm_manager=Jupyter.notebook.kernel.comm_manager;

comm_manager.register_target('{{uid}}', function(comm, msg){
    console.log(msg);
    var data = JSON.parse(msg.content.data.data);
    var code = msg.content.data.code;
    var graph = document.getElementById("{{uid}}");
    eval(code);
    Plotly.redraw(graph);
});

var send = function(msg){
    var comm = comm_manager.new_comm("{{uid}}", msg);
    comm.close();
};

require(["plotly"], function(Plotly){
    var graph = document.getElementById("{{uid}}");
    {%if onclick%}
    graph.on("plotly_click", function(data){
        if(typeof(data) == "undefined") return;
        if(!data.hasOwnProperty("lassoPoints")){
            var points = _.map(data.points, function(p){
                return {
                  curveNumber:p.curveNumber,
                  pointNumber:p.pointNumber,
                  x:p.x, y:p.y};
            });
            var send_data = {"event":"select", "data":{"select_type":"click", "points":points}};
            console.log(send_data);
            send(send_data);
        }
    });
    {% endif %}

    {%if onrelayout %}
    graph.on("plotly_relayout", function(data){
        var send_data = {"event":"relayout", "data":data};
        console.log(send_data);
        send(send_data);
    });
    {% endif %}
});

})();

</script>
""")

下面是运算核部分的Python代码，`PlotlyWidget`从`ipywidgets.HTML`继承。`HTML`类是用于显示任意HTML的控件，只需要在运算核中修改其`value`属性，即可改变页面上该控件所显示的HTML元素。

由于Plotly提供的`iplot()`会将直接生成的HTML代码发送给客户端显示，❶因此这里调用其内部生成HTML代码的函数`_plot_html()`将图表转换为HTML代码，并得到与该图表对应的唯一标识`uid`。❷然后将前面的Javascript代码添加到HTML代码之后。❸最后将最终的HTML代码赋值给`value`属性。

❹当图表上的事件响应函数中调用`send()`发送数据时，`_on_open()`方法将被调用。这里根据其`event`键判断事件的类型，调用用户设置的回调函数`click_callback`或`relayout_callback`。

❺用于发送代码和数据给客户端，这里使用`bokeh.core.json_encoder()`将数据转换为JSON字符串之后再发送。使用`json_encoder()`的好处是它可以针对NumPy的数组进行序列化。

In [3]:
from ipywidgets import HTML
from plotly.offline.offline import _plot_html
from ipykernel.comm import Comm
from bokeh.core import json_encoder

class PlotlyWidget(HTML):

    def __init__(self, figure, click_callback=None, relayout_callback=None, **kwargs):
        super().__init__(**kwargs)
        html, uid, _, _ = _plot_html(figure, False, "", True, None, None, True) #❶
        self.uid = str(uid)
        jscode = JsTemplate.render(
            uid=self.uid,
            onclick=click_callback is not None,
            onrelayout=relayout_callback is not None)
        html += jscode  #❷
        self.value = html  #❸
        self.click_callback = click_callback
        self.relayout_callback = relayout_callback
        self.comm_manager.register_target(self.uid, self._on_open) #❹

    @property
    def comm_manager(self):
        return get_ipython().kernel.comm_manager

    def _on_open(self, comm, msg):
        data = msg["content"]["data"]
        event = data["event"]
        if event == "select" and self.click_callback is not None:
            self.click_callback(data["data"])
        elif event == "relayout" and self.relayout_callback is not None:
            self.relayout_callback(data["data"])
        self.recv_msg = msg

    def send(self, code, data): #❺
        comm = Comm(target_name=str(self.uid),
                    data={"code":code, "data":json_encoder.serialize_json(data)})
        comm.close()

下面让我们使用前面的`PlotlyWidget`编写一个能动态更新图表的例子。首先创建表示图表的字典`fig`：

In [4]:
import numpy as np

x = np.linspace(0, 4*np.pi, 200)

def f(x, x0, k):
    return np.exp(-k*(x+x0)) * np.sin(4 * x + x0)

x0, k = 0, 0.1

line = {"x":x, "y":f(x, x0, k), "name":"sin*exp", 
        "line":{"width":3, "color":"blue"}}
line2 = {"x":x, "y":f(x, 0, 0), "name":"sin"}
layout = {"title": "Example", "width":600, "height":400}
fig = {"data":[line, line2], "layout":layout}

然后创建两个滑块控件`slider_x0`和`slider_k`，用于修改图表中曲线函数的参数。`selected_info`是一个`HTML`控件，用于显示图表事件响应函数发来的信息：

In [5]:
from ipywidgets import FloatSlider, HTML, VBox, HBox

slider_x0 = FloatSlider(value=x0, msg_throttle=1, description="x0:", width=200)
slider_k = FloatSlider(value=k, msg_throttle=1, min=0, max=1, description="k:", width=200)
selected_info = HTML(description="selected:", width=250, height=300)

下面创建`PlotlyWidget`对象`plot_widget`，它的第一个参数是表示图表的数据，`click_callback`和`relayout_callback`是响应图表事件的回调函数。这里的回调函数`show_data()`将接收到的数据高亮化之后显示在`selected_info`控件里。

* `click_callback`:当图表被点击时该回调函数被调用。
* `relayout_callback`:当数据显示范围改变时该回调函数被调用。

In [6]:
import json
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import HtmlFormatter   

def show_data(data):
    code = json.dumps(data, indent=2)
    selected_info.value = highlight(code, JsonLexer(), HtmlFormatter(noclasses=True))
    
plotly_widget = PlotlyWidget(fig, 
                             click_callback=show_data,
                             relayout_callback=show_data)

下面为两个滑块控件设置当属性`value`改变时的回调函数`slider_callback()`。在该函数中计算出曲线上各点Y坐标的值之后通过`send()`方法将数据和设置Y轴数据的Javascript的程序`graph.data[0]["y"] = data;`发送给客户端。

In [7]:
def slider_callback(name):
    y = f(x, slider_x0.value, slider_k.value)
    plotly_widget.send('graph.data[0]["y"] = data;', y)
    
slider_x0.observe(slider_callback, names="value")
slider_k.observe(slider_callback, names="value")

最后使用`VBox`和`HBox`显示上述控件：

In [8]:
HBox([VBox([slider_x0, slider_k, selected_info]), plotly_widget])

下面是界面截图：

![Plotly](plotly01.png)

In [ ]:
def show_py_code(fn):
    from pygments.lexers import Python3Lexer
    from IPython.display import display_html
    with open(fn, "r") as f:
        display_html(highlight(f.read(), Python3Lexer(), HtmlFormatter(noclasses=True)), raw=True)
        
show_py_code("plotlywidget.py")